In [1]:
import os
from google.colab import userdata
import requests
import hashlib
import time
import pandas as pd
from tqdm import tqdm
from pprint import pprint as pp

public_key=userdata.get('public_key')
private_key=userdata.get('private_key')


ts = str(time.time())
to_hash = ts + str(private_key) + str(public_key)
hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

characters = "https://gateway.marvel.com/v1/public/characters"
comics = "https://gateway.marvel.com/v1/public/comics"
creators = "https://gateway.marvel.com/v1/public/creators"
events = "https://gateway.marvel.com/v1/public/events"
stories = "https://gateway.marvel.com/v1/public/stories"
series = "https://gateway.marvel.com/v1/public/series"

params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": 100
}




# FUNCTIONS

In [2]:
def get_data(url, public_key, private_key, limit_num, offset_num):
  '''
  url: link de requisição da API
  public_key: chave pública gerada pela API da marvel
  private_key: chave privada gerada pela API da marvel
  limit_num: quantidade de linhas que serão pegas ao realizar requisição
  offset_num: disposição em relação a posição inicial

  return: df_normalizado: Dataframe somente com dados necessários da API
  '''


  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(url, params=params)
  data = response.json()

  df_normalizado = pd.json_normalize(data["data"]["results"])



  return df_normalizado

In [3]:
def get_data_and_total(url, public_key, private_key, limit_num, offset_num):
  '''
  url: link de requisição da API
  public_key: chave pública gerada pela API da marvel
  private_key: chave privada gerada pela API da marvel
  limit_num: quantidade de linhas que serão pegas ao realizar requisição
  offset_num: disposição em relação a posição inicial

  return: df_normalizado: Dataframe somente com dados necessários da API
  df["data"]["total"]: quantidade total de linhas na API
  '''

  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(url, params=params)
  data = response.json()
  df = pd.DataFrame(data)
  df_normalizado = pd.json_normalize(df["data"]["results"])
  return df_normalizado, df["data"]["total"]

In [4]:
def get_all_data(url):
  '''
  url: link de requisição da API
  return: retorna todos os dados da API inserida
  '''

  partial_data, total_amount = get_data_and_total(url, public_key, private_key, 100, 0)
  #Cria df de todos os dados com colunas
  all_data = pd.DataFrame(columns=partial_data.columns)

  step = 100
  total_steps = (total_amount + step - 1) // step

  offset_get_all = 0


  for offset in tqdm(range(0, total_steps * step, step), desc="Baixando dados da Marvel"):

    try:
      batch_data = get_data(url, public_key, private_key, step, offset)
      all_data = pd.concat([all_data, batch_data], ignore_index=True)
    except KeyError:
      pass

  return all_data


In [5]:
def get_all_data_faster(url):
  '''
  url: link de requisição da API
  return: retorna todos os dados da API inserida
  '''

  partial_data, total_amount = get_data_and_total(url, public_key, private_key, 100, 0)
  #Cria df de todos os dados com colunas
  all_data = pd.DataFrame(columns=partial_data.columns)

  step = 100
  total_steps = (total_amount + step - 1) // step

  offset_get_all = 0

  all_batch = []

  for offset in tqdm(range(0, total_steps * step, step), desc="Baixando dados da Marvel"):

    try:
      batch_data = get_data(url, public_key, private_key, step, offset)
      all_batch.append(batch_data)
    except (KeyError, TypeError, ValueError) as e:
      print(f"Erro no offset {offset}: {e}")
      all_batch.append(pd.DataFrame())

  all_data = pd.concat(all_batch, ignore_index=True)
  return all_data


# CHARACTERS

In [ ]:
limit_num = 100
offset_num = 0

personagens = get_data(characters, public_key, private_key, limit_num, offset_num)

In [ ]:
personagens.columns

Index(['id', 'name', 'description', 'modified', 'resourceURI', 'urls',
       'thumbnail.path', 'thumbnail.extension', 'comics.available',
       'comics.collectionURI', 'comics.items', 'comics.returned',
       'series.available', 'series.collectionURI', 'series.items',
       'series.returned', 'stories.available', 'stories.collectionURI',
       'stories.items', 'stories.returned', 'events.available',
       'events.collectionURI', 'events.items', 'events.returned'],
      dtype='object')

In [ ]:
personagens.head()

,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,21,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,61,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1010699,Aaron Stack,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,14,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2014-06-27T19:39:07+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...,jpg,55,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,68,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1


# COMICS

In [ ]:
limit_num = 100
offset_num = 0

quadrinhos = get_data(comics, public_key, private_key, limit_num, offset_num)

In [ ]:
quadrinhos.columns

Index(['id', 'digitalId', 'title', 'issueNumber', 'variantDescription',
       'description', 'modified', 'isbn', 'upc', 'diamondCode', 'ean', 'issn',
       'format', 'pageCount', 'textObjects', 'resourceURI', 'urls', 'variants',
       'collections', 'collectedIssues', 'dates', 'prices', 'images',
       'series.resourceURI', 'series.name', 'thumbnail.path',
       'thumbnail.extension', 'creators.available', 'creators.collectionURI',
       'creators.items', 'creators.returned', 'characters.available',
       'characters.collectionURI', 'characters.items', 'characters.returned',
       'stories.available', 'stories.collectionURI', 'stories.items',
       'stories.returned', 'events.available', 'events.collectionURI',
       'events.items', 'events.returned'],
      dtype='object')

In [ ]:
quadrinhos.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0


# GET ALL

## GET ALL CHARACTERS

In [ ]:
todos_personagens = get_all_data(characters)
todos_personagens

Baixando dados da Marvel: 100%|██████████| 16/16 [00:29<00:00,  1.83s/it]


,id,name,description,modified,resourceURI,urls,thumbnail.path,thumbnail.extension,comics.available,comics.collectionURI,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,1011334,3-D Man,,2014-04-29T14:18:17+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...,jpg,12,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,21,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...,jpg,4,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,7,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,7,0,https://gateway.marvel.com/v1/public/character...,[],0
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...,jpg,53,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,61,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
3,1010699,Aaron Stack,,1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,14,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,28,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2014-06-27T19:39:07+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...,jpg,55,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,68,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,1011515,Zeus,Zeus ruled a peaceful Olympus for centuries wh...,2021-08-18T22:06:56+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/f/60/4c...,png,36,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,13,43,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/character...,[],0
1560,1011127,Zodiak,"Twelve demons merged with Norman Harrison, who...",1970-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/character...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,jpg,1,https://gateway.marvel.com/v1/public/character...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,1,https://gateway.marvel.com/v1/public/character...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,0,htt

In [ ]:
#todos_personagens.to_csv("TodosPersonagens.csv")

## GET ALL COMICS

In [ ]:
todos_comics = get_all_data(comics)


Baixando dados da Marvel: 100%|██████████| 643/643 [42:20<00:00,  3.95s/it]


In [ ]:
todos_comics.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0


In [ ]:
#todos_comics.to_csv("TodosComics.csv")

In [ ]:
todos_comics

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,82967,0,Marvel Previews (2017),0,,,2019-11-07T08:46:15+0000,,75960608839302811,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
1,82965,0,Marvel Previews (2017),0,,,2019-08-21T17:11:27+0000,,75960608839302611,JUL190068,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
2,82970,52952,Marvel Previews (2017),0,,,2020-02-07T09:35:32+0000,,75960608839303111,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/82...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/82...,[],0
3,61137,0,The Punisher (2016) #5 (Cosplay Variant),5,Cosplay Variant,None,2016-09-01T12:40:16+0000,,75960608413500521,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/61...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/61...,[],0
4,37497,0,MARVELS (1994) #1,1,,None,2014-05-08T12:18:55+0000,,,,...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0,0,https://gateway.marvel.com/v1/public/comics/37...,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64263,21671,0,Psi-Force (1986) #28,28,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0
64264,21672,0,Psi-Force (1986) #29,29,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0
64265,21674,0,Psi-Force (1986) #30,30,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0
64266,21675,0,Psi-Force (1986) #31,31,,None,-0001-11-30T00:00:00+0000,,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/21...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/21...,[],0


## GET ALL CREATORS

In [ ]:
todos_creators = get_all_data(creators)

Baixando dados da Marvel: 100%|██████████| 66/66 [01:25<00:00,  1.29s/it]


In [ ]:
todos_creators

,id,firstName,middleName,lastName,suffix,fullName,modified,resourceURI,urls,thumbnail.path,...,series.items,series.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,12728,,,,,,2015-04-29T16:22:30+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0
1,12725,,,,,,2015-04-29T16:22:30+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0
2,12727,,,,,,2015-04-29T16:22:30+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0
3,12733,,,,,,2015-04-29T16:22:30+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0
4,12788,,,,,,2015-11-23T17:40:17+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5327,14364,Chrissie,,Zullo,,Chrissie Zullo,2025-05-08T09:19:55+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,46,https://gateway.marvel.com/v1/public/creators/...,[{'resourceURI': 'https://gateway.marvel.com/v...,20,0,https://gateway.marvel.com/v1/public/creators/...,[],0
5328,3697,Joel,,Zulueta,,Joel Zulueta,2007-01-01T00:00:00+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,1,https://gateway.marvel.com/v1/public/creators/...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,0,https://gateway.marvel.com/v1/public/creators/...,[],0
5329,6505,Antony,De,Zuniga,,Antony De Zuniga,2010-11-23T09:53:50+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0,0,https://gateway.marvel.com/v1/public/creators/...,[],0
5330,14035,Christina,,Zuniga,,Christina Z,2025-04-19T10:40:28+0000,https://gateway.marvel.com/v1/public/creators/...,"[{'type': 'detail', 'url': 'http://marvel.com/...",http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,9,https://gateway.marvel.com/v1/public/creators/...,[{'resourceURI': 'https://gateway.marvel.com/v...,9,0,https://gateway.marvel.com/v1/public/creators/...,[],0


In [ ]:
#todos_creators.to_csv("TodosCreators.csv")

## GET ALL EVENTS

In [ ]:
todos_events = get_all_data(events)
todos_events.to_csv("TodosEvents.csv")

KeyError: 'data'

## GET ALL STORIES

In [ ]:
todos_stories = get_all_data_faster(stories)
todos_stories.to_csv("TodosStories.csv")

Baixando dados da Marvel:  51%|█████     | 748/1481 [1:26:59<1:16:10,  6.23s/it]

Erro no offset 74700: 'data'


Baixando dados da Marvel:  51%|█████     | 749/1481 [1:26:59<54:00,  4.43s/it]  

Erro no offset 74800: 'data'


Baixando dados da Marvel:  51%|█████     | 750/1481 [1:26:59<38:30,  3.16s/it]

...

Erro no offset 147800: 'data'


Baixando dados da Marvel: 100%|█████████▉| 1480/1481 [1:29:19<00:00,  5.07it/s]

Erro no offset 147900: 'data'


Baixando dados da Marvel: 100%|██████████| 1481/1481 [1:29:20<00:00,  3.62s/it]

Erro no offset 148000: 'data'


## GET ALL SERIES

In [ ]:
todos_series = get_all_data(series)
todos_series.to_csv("TodosSeries.csv")

Baixando dados da Marvel: 100%|██████████| 158/158 [04:26<00:00,  1.69s/it]


# GET SPECIFIC STORIES

In [58]:
def get_all_data_specific(url, id, specific_path):
  '''
  url: link de requisição da API
  return: retorna todos os dados da API inserida
  '''

  partial_data, total_amount = get_data_and_total(f"{url}/{id}/{specific_path}", public_key, private_key, 100, 0)
  #Cria df de todos os dados com colunas
  all_data = pd.DataFrame(columns=partial_data.columns)

  step = 100
  total_steps = (total_amount + step - 1) // step

  offset_get_all = 0


  for offset in tqdm(range(0, total_steps * step, step), desc="Baixando dados da Marvel"):

    try:
      batch_data = get_data(f"{url}/{id}/{specific_path}", public_key, private_key, step, offset)
      all_data = pd.concat([all_data, batch_data], ignore_index=True)
    except KeyError:
      pass

  return all_data


In [21]:
def get_specific(url, id, public_key, private_key, limit_num, offset_num, specific_path):
  '''
  url: link de requisição da API
  public_key: chave pública gerada pela API da marvel
  private_key: chave privada gerada pela API da marvel
  limit_num: quantidade de linhas que serão pegas ao realizar requisição
  offset_num: disposição em relação a posição inicial

  return: df_normalizado: Dataframe somente com dados necessários da API
  '''


  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(f"{url}/{id}/{specific_path}", params=params)
  data = response.json()
  print(data)
  df_normalizado = pd.json_normalize(data["data"]["results"])



  return df_normalizado

In [35]:
def get_data_and_total_specific(url, public_key, private_key, limit_num, offset_num, id, specific_path):
  '''
  url: link de requisição da API
  public_key: chave pública gerada pela API da marvel
  private_key: chave privada gerada pela API da marvel
  limit_num: quantidade de linhas que serão pegas ao realizar requisição
  offset_num: disposição em relação a posição inicial

  return: df_normalizado: Dataframe somente com dados necessários da API
  df["data"]["total"]: quantidade total de linhas na API
  '''

  ts = str(time.time())
  to_hash = ts + str(private_key) + str(public_key)
  hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

  params = {
    "ts": ts,
    "apikey": public_key,
    "hash": hash_md5,
    "limit": limit_num,
    "offset": offset_num
  }

  response = requests.get(f"{url}/{id}/{specific_path}", params=params)
  print(f"response url: {response.url}")
  data = response.json()

  df = pd.DataFrame(data)
  df_normalizado = pd.json_normalize(df["data"]["results"])
  return df_normalizado, df["data"]["total"]

In [32]:
def get_all_data_faster_specific(url, public_key, private_key, limit_num, offset_num, id, specific_path):
  '''
  url: link de requisição da API
  return: retorna todos os dados da API inserida
  '''

  partial_data, total_amount = get_data_and_total_specific(url, public_key, private_key, 100, 0, id, specific_path)
  print(total_amount)
  #Cria df de todos os dados com colunas
  all_data = pd.DataFrame(columns=partial_data.columns)

  step = 100
  total_steps = (total_amount + step - 1) // step

  offset_get_all = 0

  all_batch = []

  for offset in tqdm(range(0, total_steps * step, step), desc="Baixando dados da Marvel"):

    try:
      batch_data = get_specific(url, public_key, private_key, step, offset, id, specific_path)
      all_batch.append(batch_data)
    except (KeyError, TypeError, ValueError) as e:
      print(f"Erro no offset {offset}: {e}")
      all_batch.append(pd.DataFrame())

  if len(all_batch) == 0:
    return pd.DataFrame()
  else:
    all_data = pd.concat(all_batch, ignore_index=True)
    return all_data


In [29]:
specific_story = "https://gateway.marvel.com/v1/public/creators"
#df = get_stories(specific_story, 1680, public_key, private_key, 100, 0)

In [30]:
df = pd.DataFrame()
len(df)

0

In [48]:
id = 726

In [49]:
partial_data, total_amount = get_data_and_total_specific(specific_story, public_key, private_key, 100, 0, id, "comics")

response url: https://gateway.marvel.com/v1/public/creators/726/comics?ts=1747835392.8425288&apikey=8ed993fa80600a97b969f62b58729d0c&hash=4f4870800a7a9a381e91a5cd4a46a6f1&limit=100&offset=0


In [ ]:
partial_data

In [ ]:
list_of_ids = [5282,  955,  726, 3791, 3645, 1290, 3112, 3403, 2848, 4210, 5161,  451,
       2998, 2941, 5160, 4513, 1462, 1362, 3196,  515, 1680,  799,  208, 1669,
       1502]

i = 0
for item in list_of_ids:
  df = get_all_data_faster_specific(specific_story, public_key, private_key, 100, 0, item, "comics")
  if len(df) == 0:
    i += 1
    continue
  else:
    i += 1
    break

for item in list_of_ids[i:]:
  df = get_all_data_faster_specific(specific_story, public_key, private_key, 100, 0, item, "comics")
  if len(df) == 0:
    continue
  else:
    df = pd.concat(df, ignore_index=True)

In [89]:
df2= get_all_data_specific(specific_story, 14311, "comics")

Baixando dados da Marvel: 100%|██████████| 11/11 [00:15<00:00,  1.39s/it]


In [75]:
df2.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,116255,0,DARKHOLD: THE SAGA OF THE BOOK OF SINS (Trade ...,1,,"Collects Marvel Spotlight (1971) #4, Werewolf ...",2024-07-25T04:12:57+0000,978-1-302-96023-0,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/11...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/11...,[],0
1,104954,0,Marvel Masterworks: Werewolf By Night Vol. 2 (...,0,,"Collects Werewolf by Night (1972) #9-21, Tomb ...",2023-11-14T09:35:32+0000,978-1-302-94948-8,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/10...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/10...,[],0
2,108829,0,The Avengers Omnibus Vol. 5 (Hardcover),0,,"Collects Avengers (1963) #120-149, Giant-Size ...",2023-09-22T12:07:14+0000,978-1-302-95411-6,,,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,2,https://gateway.marvel.com/v1/public/comics/10...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/10...,[],0
3,97227,60611,Marvel Masterworks: The Tomb Of Dracula Vol. 2...,0,,"Collects Tomb of Dracula (1972) #12-22, Werewo...",2022-11-14T09:39:01+0000,978-1-302-93342-5,,,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,2,https://gateway.marvel.com/v1/public/comics/97...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/97...,[],0
4,97231,60240,Marvel Masterworks: Werewolf By Night Vol. 1 (...,0,,"Collects Marvel Spotlight (1971) #2-4, Werewol...",2022-09-27T13:14:38+0000,978-1-302-93346-3,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/97...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/97...,[],0


In [66]:

for item in list_of_ids[2:]:
  aux = get_all_data_specific(specific_story, item, "comics")
  df = pd.concat([df, aux], ignore_index=True)

Baixando dados da Marvel: 100%|██████████| 1/1 [00:00<00:00,  1.61it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 100%|██████████| 2/2 [00:02<00:00,  1.30s/it]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 100%|██████████| 2/2 [00:02<00:00,  1.33s/it]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 100%|██████████| 1/1 [00:00<00:00,  1.56it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]
Baixando dados da Marvel: 0it [00:00, ?it/s]
Baixando dados da Marvel: 100%|██████████| 6/6 [00:09<00:00,  1.59s/it]
B

In [76]:
df.head()
#df.to_csv("Creators_Especifico.csv")

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,116255,0,DARKHOLD: THE SAGA OF THE BOOK OF SINS (Trade ...,1,,"Collects Marvel Spotlight (1971) #4, Werewolf ...",2024-07-25T04:12:57+0000,978-1-302-96023-0,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/11...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/11...,[],0
1,104954,0,Marvel Masterworks: Werewolf By Night Vol. 2 (...,0,,"Collects Werewolf by Night (1972) #9-21, Tomb ...",2023-11-14T09:35:32+0000,978-1-302-94948-8,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/10...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/10...,[],0
2,108829,0,The Avengers Omnibus Vol. 5 (Hardcover),0,,"Collects Avengers (1963) #120-149, Giant-Size ...",2023-09-22T12:07:14+0000,978-1-302-95411-6,,,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,2,https://gateway.marvel.com/v1/public/comics/10...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/10...,[],0
3,97227,60611,Marvel Masterworks: The Tomb Of Dracula Vol. 2...,0,,"Collects Tomb of Dracula (1972) #12-22, Werewo...",2022-11-14T09:39:01+0000,978-1-302-93342-5,,,...,[{'resourceURI': 'https://gateway.marvel.com/v...,1,2,https://gateway.marvel.com/v1/public/comics/97...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/97...,[],0
4,97231,60240,Marvel Masterworks: Werewolf By Night Vol. 1 (...,0,,"Collects Marvel Spotlight (1971) #2-4, Werewol...",2022-09-27T13:14:38+0000,978-1-302-93346-3,,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/97...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/97...,[],0


In [79]:
df2.columns

Index(['id', 'digitalId', 'title', 'issueNumber', 'variantDescription',
       'description', 'modified', 'isbn', 'upc', 'diamondCode', 'ean', 'issn',
       'format', 'pageCount', 'textObjects', 'resourceURI', 'urls', 'variants',
       'collections', 'collectedIssues', 'dates', 'prices', 'images',
       'series.resourceURI', 'series.name', 'thumbnail.path',
       'thumbnail.extension', 'creators.available', 'creators.collectionURI',
       'creators.items', 'creators.returned', 'characters.available',
       'characters.collectionURI', 'characters.items', 'characters.returned',
       'stories.available', 'stories.collectionURI', 'stories.items',
       'stories.returned', 'events.available', 'events.collectionURI',
       'events.items', 'events.returned'],
      dtype='object')

In [88]:
df2.head()

,id,digitalId,title,issueNumber,variantDescription,description,modified,isbn,upc,diamondCode,...,characters.items,characters.returned,stories.available,stories.collectionURI,stories.items,stories.returned,events.available,events.collectionURI,events.items,events.returned
0,59739,0,Civil War II: Kingpin (2016) #1 (Noto Characte...,1,Noto Character Variant,None,2016-06-29T17:03:12+0000,,75960608531600131,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/59...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/59...,[],0
1,66861,47755,Poe Dameron (2016) #26,26,,THE AWAKENING BEGINS! Lor San Tekka has been f...,2021-08-30T14:36:07+0000,,75960608458602611,,...,[],0,2,https://gateway.marvel.com/v1/public/comics/66...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/66...,[],0
2,66537,47578,Damnation: Johnny Blaze - Ghost Rider (2018) #1,1,,DAMNATION TIE-IN! Johnny Blaze takes the fight...,2022-05-25T17:50:56+0000,,759606089391000111,JAN180944,...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,2,https://gateway.marvel.com/v1/public/comics/66...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/66...,[],0
3,66558,47595,Poe Dameron (2016) #25,25,,It’s the end of an era for Poe Dameron. With t...,2021-08-30T14:36:06+0000,,75960608458602511,JAN181022,...,[],0,2,https://gateway.marvel.com/v1/public/comics/66...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/66...,[],0
4,66269,47321,Black Panther (2016) #170,170,,"With Storm back by his side, T'Challa finally ...",2020-11-18T10:55:58+0000,,75960608767917011,,...,[{'resourceURI': 'https://gateway.marvel.com/v...,3,2,https://gateway.marvel.com/v1/public/comics/66...,[{'resourceURI': 'https://gateway.marvel.com/v...,2,0,https://gateway.marvel.com/v1/public/comics/66...,[],0


In [86]:
df2["creators.items"][0]

[{'resourceURI': 'https://gateway.marvel.com/v1/public/creators/2133',
  'name': 'Tom Brevoort',
  'role': 'editor'},
 {'resourceURI': 'https://gateway.marvel.com/v1/public/creators/12376',
  'name': 'Wilson Moss',
  'role': 'editor'},
 {'resourceURI': 'https://gateway.marvel.com/v1/public/creators/5160',
  'name': 'Philip J. Noto',
  'role': 'penciller (cover)'}]